## Práctica 2. Procesamiento de lenguaje natural

Grupo 16: Adina Han y Diego Ambite

### Parte 1. Análisis de sentimientos

In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
with open("yelp_labelled.txt","r") as text_file:
    lines_org = text_file.read().split('\n')
len(lines_org)

In [ ]:
lines = [line.split("\t") for line in lines_org if len(line.split("\t"))==2 and line.split("\t")[1]!='']
len(lines)

In [ ]:
data_line = np.array(lines)
text = data_line[:,0]
target = data_line[:,1]

In [ ]:
data = {'Texto':text,'Target':target}


In [ ]:
df = pd.DataFrame(data=data)
df.head()

In [ ]:
df['Label'] = df.Target.map({'0':'Mala','1':'Buena'})

In [ ]:
df.head()

In [ ]:
df.keys()

In [ ]:
df['message_len'] = df.Texto.apply(len)
df.head()

In [ ]:
plt.figure(figsize=(12, 8))

df[df.Label=='Mala'].message_len.plot(bins=35, kind='hist', color='blue', 
                                       label='Bad messages', alpha=0.6)
df[df.Label=='Buena'].message_len.plot(kind='hist', color='red', 
                                       label='Good messages', alpha=0.6)
plt.legend()
plt.xlabel("Message Length")

In [ ]:
df[df.Label=='Mala'].describe()

In [ ]:
df[df.Label=='Buena'].describe()

### Apartado a)

#### Configura una partición train-test usando el 75% de los datos para entrenamiento y el 25% restante para test.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Texto'], df['Target'], test_size=0.25, random_state=333)


#### Vamos a estudiar varias representaciones de bolsa de palabras, pero todas ellas utilizarán countVectorizer con el diccionario que se crea a partir de los términos del propio corpus y la lista de palabras vacías (stop_words) que proporciona sklearn para el inglés. Las 4 posibilidades que estudiaremos surgen de combinar los siguientes 2 parámetros:

    - Bolsa de palabras binaria (usando el countVectorizer con el parámetro binary=True y sin usar TfidfTransformer) y bolsa de palabras con TF/IDF (usando primero el countVectorizer con el parámetro binary=False, y sobre el resultado el TfidfTransformer)
    
    - Usando un rango de n-gramas de (1,1) y de (1,2) (parámetro ngram_range del countVectorizer). Es decir, haciendo que la bolsa de palabras se consideren solamente monogramas, o que se consideren monogramas y bigramas


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
vectorizer_binary_ngram11 = CountVectorizer(stop_words='english',binary=True, ngram_range=(1,1))
# Tomamos los textos del conjunto de entrenamiento y los transformamos en 
# una matriz de datos (palabras) según el diccionario estándar
X_train_binary_ngram11=vectorizer_binary_ngram11.fit_transform(X_train)


In [ ]:
X_test_binary_ngram11 = vectorizer_binary_ngram11.transform(X_test)

In [ ]:
X_train_binary_ngram11.shape

In [ ]:
print(X_train_binary_ngram11)

In [ ]:
vectorizer_binary_ngram11.vocabulary_
 

In [ ]:
feature_names = vectorizer_binary_ngram11.get_feature_names()

print(len(feature_names))
print(feature_names)
#print(train_vector_data[10])

In [ ]:
vectorizer_tfidf_ngram11 = CountVectorizer(stop_words='english',binary=False, ngram_range=(1,1))
X_train_tfidf_ngram11=vectorizer_tfidf_ngram11.fit_transform(X_train)

In [ ]:
X_test_tfidf_ngram11=vectorizer_tfidf_ngram11.transform(X_test)

In [ ]:
vectorizer_tfidf_ngram11.vocabulary_

In [ ]:
feature_names_tfidf = vectorizer_tfidf_ngram11.get_feature_names()

print(len(feature_names_tfidf))
print(feature_names_tfidf)
#print(train_vector_data[10])

In [ ]:
# Calculamos el valor TF-IDF 
tfidfer = TfidfTransformer()
train_tfidf_preprocessed = tfidfer.fit_transform(X_train_tfidf_ngram11)
test_tfidf_preprocessed = tfidfer.transform(X_test_tfidf_ngram11)

In [ ]:
vectorizer_binary_ngrams12 = CountVectorizer(stop_words='english',binary=True, ngram_range=(1,2))
X_train_binary_ngrams12 = vectorizer_binary_ngrams12.fit_transform(X_train)

In [ ]:
X_test__binary_ngrams12 = vectorizer_ngrams12.transform(X_test)

In [ ]:
feature_names_ngrams12 = vectorizer_binary_ngrams12.get_feature_names()

print(len(feature_names_ngrams12))
print(feature_names_ngrams12)
#print(train_vector_data[10])

In [ ]:
vectorizer_tfidf_ngrams12 = CountVectorizer(stop_words='english',binary=False, ngram_range=(1,2))
X_train_tfidf_ngrams12 = vectorizer_tfidf_ngrams12.fit_transform(X_train)


In [ ]:
X_test_ifidf_ngrams12 = vectorizer_tfidf_ngrams12.transform(X_test)

In [ ]:
tfidfer = TfidfTransformer()
train_tfidf_preprocessed = tfidfer.fit_transform(X_train_tfidf_ngrams12)
test_tfidf_preprocessed = tfidfer.transform(X_test_ifidf_ngrams12)

In [ ]:
import numpy as np
import numpy.ma as ma

def write_terms (feature_names, data, vector_data, index):
    '''
    Escribe los términos presentes en un mensaje representado como bolsa de palabras.
    
    - feature_names: terminos usados para vectorizar
    - data: lista de mensajes original (si data==None no se muestra el mensaje original)
    - vector_data: matriz (dispersa) de mensaje vectorizados
    - index: posición del mensaje a mostrar
    '''
    # máscara para seleccionar sólo el mensaje en posición index
    mask=vector_data[index,:]>0
    
    # términos que aparecen en ese mensaje vectorizado
    terminos = ma.array(feature_names, mask = ~(mask[0].toarray()))
    
    # mostrar mensaje original
    if data:
        print('Mensaje', index, ':', data[index])
    
    # mostrar términos que aparecen en el mensaje vectorizado
    print('Mensaje', index, 'vectorizado:', terminos.compressed(),'\n')

In [ ]:
#write_terms(feature_names, train_data.data, train_vector_data, 10)
write_terms(feature_names, None, X_train_binary, 0)
write_terms(feature_names, None, X_train_tfidf, 0)
write_terms(feature_names, None, X_train_ngrams11, 0)

write_terms(feature_names, None, X_train_binary, 10)
write_terms(feature_names, None, X_train_tfidf, 10)
write_terms(feature_names, None, X_train_ngrams11, 10)

write_terms(feature_names, None, X_train_binary, 100)
write_terms(feature_names, None, X_train_tfidf, 100)
write_terms(feature_names, None, X_train_ngrams11, 100)




    
#### Para cada una de esas 4 combinaciones entrenaremos dos clasificadores:

     1. Un clasificador naive bayes, eligiendo el más adecuado para cada caso.
     
     2. Un árbol de decisión buscando un valor óptimo para uno de los siguientes parámetros para que se maximice la tasa de aciertos en el conjunto de test: max_depth, min_samples_leaf o max_leaf_nodes (siempre el mismo).
     


In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(X_train_binary, y_train)

mnb_train_predictions = mnb_classifier.predict(X_train_binary)
mnb_test_predictions = mnb_classifier.predict(X_test_binary)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(train_tfidf_preprocessed, y_train)

mnb_train_predictions = mnb_classifier.predict(train_tfidf_preprocessed)
mnb_test_predictions = mnb_classifier.predict(test_tfidf_preprocessed)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(train_ngrams11_preprocessed, y_train)

mnb_train_predictions = mnb_classifier.predict(train_ngrams11_preprocessed)
mnb_test_predictions = mnb_classifier.predict(test_ngrams11_preprocessed)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(X_train_ngrams11, y_train)

mnb_train_predictions = mnb_classifier.predict(X_train_ngrams11)
mnb_test_predictions = mnb_classifier.predict(X_test_ngrams11)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(X_train_ngrams12, y_train)

mnb_train_predictions = mnb_classifier.predict(X_train_ngrams12)
mnb_test_predictions = mnb_classifier.predict(X_test_ngrams12)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(train_ngrams12_preprocessed, y_train)

mnb_train_predictions = mnb_classifier.predict(train_ngrams12_preprocessed)
mnb_test_predictions = mnb_classifier.predict(test_ngrams12_preprocessed)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

## ¿¿¿¿¿¿¿¿¿¿¿¿¿¿hay que usar tfidftransform() con las n-grams??????????????? 

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(X_train_binary, y_train)

tree_train_predictions = tree_classifier.predict(X_train_binary)
tree_test_predictions = tree_classifier.predict(X_test_binary)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

In [ ]:
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(train_tfidf_preprocessed, y_train)

tree_train_predictions = tree_classifier.predict(train_tfidf_preprocessed)
tree_test_predictions = tree_classifier.predict(test_tfidf_preprocessed)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

In [ ]:
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(train_ngrams11_preprocessed, y_train)

tree_train_predictions = tree_classifier.predict(train_ngrams11_preprocessed)
tree_test_predictions = tree_classifier.predict(test_ngrams11_preprocessed)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

In [ ]:
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(X_train_ngrams11, y_train)

tree_train_predictions = tree_classifier.predict(X_train_ngrams11)
tree_test_predictions = tree_classifier.predict(X_test_ngrams11)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

In [ ]:
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(X_train_ngrams12, y_train)

tree_train_predictions = tree_classifier.predict(X_train_ngrams12)
tree_test_predictions = tree_classifier.predict(X_test_ngrams12)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

In [ ]:
from sklearn import tree
import numpy as np

# Creamos el clasificador con los valores por defecto
tree_classifier = tree.DecisionTreeClassifier()
tree_classifier.fit(train_ngrams12_preprocessed, y_train)

tree_train_predictions = tree_classifier.predict(train_ngrams12_preprocessed)
tree_test_predictions = tree_classifier.predict(test_ngrams12_preprocessed)

print("Árbol, porcentaje de aciertos en entrenamiento:", np.mean(tree_train_predictions == y_train))
print("Árbol, porcentaje de aciertos en test:", np.mean(tree_test_predictions == y_test))

## ¿¿¿¿¿¿¿¿¿¿es normal que en los arboles tengamos el mismo porcentaje de aciertos de entrenamiento??????????

## ¿¿¿¿¿de los arboles de arriba tenemos que elegir el que tenga mejor porcentaje de aciertos en test????????????

## ¿¿¿¿¿¿¿¿¿¿¿¿¿¿¿que valor elegir para max_dept, hasta que punto podemos incrementar este valor ????????????????????????

#### Analiza la tasa de aciertos de entrenamiento y test de los 2 clasificadores en las 4 representaciones de bolsa de palabras (8 configuraciones en total) y contesta a las siguientes preguntas:

    - ¿Hay un clasificador que sea superior al otro? ¿por qué crees que sucede?
    
    - Para cada clasificador, ¿tiene un efecto positivo el añadir “complejidad” a la vectorización? Es decir, añadir bigramas y añadir tf-idf. ¿Por qué crees que sucede este efecto positivo o la falta del mismo? 
    


Los resultados que obtenemos con el arbol de decisión son mejores que los obtenidos con el clasificador de naive bayes. MIRAR ARBOLES DE DECISION Y NAIVE BAYES (para contestar por que creemos que sucede eso)

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
train_accuracy = []
test_accuracy = []

max_depths = range(1, 40)
for md in max_depths: 
    # Repetimos el proceso pero modificando los parámetros de aprendizaje del árbol
    clf = DecisionTreeClassifier(criterion="entropy",  # por defecto Gini pero podemos cambiar a entropía
                                 max_depth=md,          # profundidad máxima del árbol
                                 min_samples_split=5,  # mínimo de muestras en el nodo para seguir dividiéndolo
                                 random_state=333)
    clf = clf.fit(train_ngrams12_preprocessed, y_train)
        
    #train_accuracy.append(np.mean(scores['train_score']))
    #test_accuracy.append(np.mean(scores['test_score']))
    # Calculamos la precisión del modelo de entrenamiento y de test
    train_accuracy.append(clf.score(train_ngrams12_preprocessed, y_train))
    test_accuracy.append(clf.score(test_ngrams12_preprocessed, y_test))
    plt.figure(figsize=(50,50))
    plot_tree(clf, filled=True, feature_names=feature_names_ngrams12, class_names=df['Label'], rounded=True)
    plt.show()
train_accuracy, test_accuracy



In [ ]:
import matplotlib.pyplot as plt

# Draw lines
plt.plot(max_depths, train_accuracy, color="r",  label="Training")
plt.plot(max_depths, test_accuracy, color="g", label="Test")

# Create plot
plt.title("Curva de aprendizaje")
plt.xlabel("Parametro"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show() 

In [ ]:
train_accuracy = []
test_accuracy = []

min_samples_splits = range(2, 10)
for mss in min_samples_splits: 
    # Repetimos el proceso pero modificando los parámetros de aprendizaje del árbol
    clf = DecisionTreeClassifier(criterion="entropy",  # por defecto Gini pero podemos cambiar a entropía
                                 max_depth=5,          # profundidad máxima del árbol
                                 min_samples_split=mss,  # mínimo de muestras en el nodo para seguir dividiéndolo
                                 random_state=333)
    clf = clf.fit(train_ngrams12_preprocessed, y_train)
        
    #train_accuracy.append(np.mean(scores['train_score']))
    #test_accuracy.append(np.mean(scores['test_score']))
    # Calculamos la precisión del modelo de entrenamiento y de test
    train_accuracy.append(clf.score(train_ngrams12_preprocessed, y_train))
    test_accuracy.append(clf.score(test_ngrams12_preprocessed, y_test))
    plt.figure(figsize=(50,50))
    plot_tree(clf, filled=True, feature_names=feature_names_ngrams12, class_names=df['Label'], rounded=True)
    plt.show()
train_accuracy, test_accuracy

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Vamos a mostrar el árbol de decisión generado usando plot_tree
plt.figure(figsize=(50,50))
plot_tree(clf, filled=True, feature_names=feature_names_ngrams12, class_names=df['Label'], rounded=True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Draw lines
plt.plot(min_samples_splits, train_accuracy, color="r",  label="Training")
plt.plot(min_samples_splits, test_accuracy, color="g", label="Test")

# Create plot
plt.title("Curva de aprendizaje")
plt.xlabel("Parametro"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show() 

In [ ]:
train_accuracy = []
test_accuracy = []

min_samples_leafs = range(3, 15)
for msl in min_samples_leafs: 
    # Repetimos el proceso pero modificando los parámetros de aprendizaje del árbol
    clf = DecisionTreeClassifier(criterion="entropy",  # por defecto Gini pero podemos cambiar a entropía
                                 max_depth=5,          # profundidad máxima del árbol
                                 min_samples_split=4,  # mínimo de muestras en el nodo para seguir dividiéndolo
                                 min_samples_leaf=msl,
                                 random_state=333)
    clf = clf.fit(train_ngrams12_preprocessed, y_train)
        
    #train_accuracy.append(np.mean(scores['train_score']))
    #test_accuracy.append(np.mean(scores['test_score']))
    # Calculamos la precisión del modelo de entrenamiento y de test
    train_accuracy.append(clf.score(train_ngrams12_preprocessed, y_train))
    test_accuracy.append(clf.score(test_ngrams12_preprocessed, y_test))
    plt.figure(figsize=(50,50))
    plot_tree(clf, filled=True, feature_names=feature_names_ngrams12, class_names=df['Label'], rounded=True)
    plt.show()
train_accuracy, test_accuracy



In [ ]:
import matplotlib.pyplot as plt

# Draw lines
plt.plot(min_samples_leafs, train_accuracy, color="r",  label="Training")
plt.plot(min_samples_leafs, test_accuracy, color="g", label="Test")

# Create plot
plt.title("Curva de aprendizaje")
plt.xlabel("Parametro"), plt.ylabel("Accuracy Score"), plt.legend(loc="best")
plt.tight_layout()
plt.show() 

In [ ]:
# Pintando árboles con sckit-learn

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

# Vamos a mostrar el árbol de decisión generado usando plot_tree
plt.figure(figsize=(50,50))
plot_tree(clf, filled=True, feature_names=feature_names_ngrams12, class_names=df['Label'], rounded=True)
plt.show()

In [ ]:
from sklearn.model_selection import cross_validate
import numpy as np


clf = DecisionTreeClassifier(criterion="entropy", 
                              max_depth=14, 
                              min_samples_split=4,
                              min_samples_leaf=4,
                              random_state=333)

clf = clf.fit(train_ngrams12_preprocessed, y_train)
    
train_accuracy = clf.score(train_ngrams12_preprocessed, y_train)
test_accuracy = clf.score(test_ngrams12_preprocessed, y_test)
train_accuracy, test_accuracy

#### Selecciona el mejor árbol de decisión y obtén las 25 variables con más poder discriminante:
    
    - ¿Predominan más las palabras de uno u otro sentimiento? ¿por qué? ¿hay ruido? 
    


In [ ]:
def print_top20_features_in_trees(vectorizer, clf):
    """Prints features with the highest coefficient values"""
    feature_names = vectorizer_ngrams12.get_feature_names()
    
    top20 = np.argsort(clf.feature_importances_)[-20:]
    reversed_top = top20[::-1]
    print("Top 20 features in the tree\n")
    print("%s" % ( " / ".join(feature_names[j] for j in reversed_top)))

In [ ]:
print_top20_features_in_trees(vectorizer_ngrams12,tree_classifier)

#### Selecciona el mejor clasificador naive bayes y obtén las 25 variables con más presencia en cada clase:
    
    - ¿Tienen sentido las palabras seleccionadas? ¿hay ruido (palabras sin sentimiento o de sentimiento opuesto al esperado)? ¿por qué crees que suceden estos fenómenos?
    


In [ ]:
from sklearn.naive_bayes import MultinomialNB

mnb_classifier = MultinomialNB()

mnb_classifier.fit(X_train_ngrams12, y_train)

mnb_train_predictions = mnb_classifier.predict(X_train_ngrams12)
mnb_test_predictions = mnb_classifier.predict(X_test_ngrams12)

print("Multinomial Naive Bayes, porcentaje de aciertos en entrenamiento:", np.mean(mnb_train_predictions == y_train))
print("Multinomial Naive Bayes, porcentaje de aciertos en test:", np.mean(mnb_test_predictions == y_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# NOTA: Aquí elegimos analizar un determinado clasificador y sus predicciones 
# Por ejemplo el naive bayes

classifier=mnb_classifier
predictions = mnb_test_predictions
target_names={'Bad','Good'}
print(classification_report(y_test, predictions, target_names=target_names))

In [ ]:
def print_top25_features_per_class_in_NB(vectorizer, clf, class_labels):
    """Prints features with the highest coefficient values, per class"""
    feature_names = vectorizer_ngrams12.get_feature_names()
    print("Top 25 features per class\n")
    for i, class_label in enumerate(class_labels):
        top25 = np.argsort(clf.feature_log_prob_[i])[-25:]
        reversed_top = top25[::-1]
        
        print("%s: %s" % (class_label,
              " / ".join(feature_names[j] for j in reversed_top)),'\n')
        
        #Descomentar para ver el índice de los términos en el diccionario
        #print("%s " % (" / ".join(str(j) for j in reversed_top)),'\n')

In [ ]:
print_top25_features_per_class_in_NB(vectorizer_ngrams12,mnb_classifier,target_names) 

#### Finalmente, explica de manera razonada las conclusiones que has extraído de todo el estudio realizado en este apartado.






### Apartado b)

#### Toma el mejor clasificador Naive Bayes y el mejor árbol de decisión y analiza a fondo sus resultados en el conjunto de test.

   

#### 1. Analiza la precisión y la exhaustividad de cada clasificador en cada una de las clases (opiniones positivas y negativas).
    
         Para cada clasificador, ¿tiene un comportamiento homogéneo a la hora de clasificar ambas clases?
         ¿Cuáles son las fortalezas y debilidades de cada uno de los clasificadores?
         ¿Hay algún clasificador que sea mejor que el otro en todo?
         ¿Coinciden ambos clasificadores a la hora de clasificar mejor una clase que la otra?
         


#### 2. Pinta los 8 primeros niveles del árbol de decisión y comenta lo que ves.
         ¿Qué estructura tiene el árbol?
         ¿Cómo interpretas los niveles que has pintado? ¿tienen algún sentido con respecto a la tasa de aciertos, o la precisión y exhaustividad del clasificador? o ¿Hay nodos impuros?
         


#### 3. Por cada clasificador identifica 2 críticas que hayan sido falsas positivas (malas críticas calificadas como buenas) y 2 críticas que han sido falsas negativas (buenas críticas clasificadas como malas). Analiza tanto su texto original, como el vector de palabras resultante (solamente los términos activos).
         ¿Por qué crees que ha fallado el clasificador en cada uno de los casos?
         ¿Se te ocurre alguna idea sobre cómo mejorar el clasificador de sentimiento?